In [48]:
import json
from collections import defaultdict

In [50]:
# Read words in to a dictionary
file = open('AFINN.txt')
word_dict = defaultdict(int)
for line in file.readlines():
    pair = line.rstrip().split("\t")
    word = pair[0]
    score = int(pair[1])
    word_dict[word] = score

In [68]:
for key in word_dict.keys():
    if len(key.split(' '))>1:
        print(key)

can't stand
cashing in
cool stuff
does not work
dont like
fed up
green wash
green washing
messing up
no fun
not good
not working
right direction
screwed up
some kind


In [78]:

for word in word_dict.keys():
    if '-' in word:
        print(word)

cover-up
made-up
once-in-a-lifetime
self-confident
self-deluded
short-sighted
short-sightedness
son-of-a-bitch


In [79]:
# Load melbGrid
location_list = [] # Store grid information
with open('melbGrid.json', 'r') as f:
    grid_dic = json.load(f)
    for location_dict in grid_dic['features']:
        location_list.append(location_dict['properties'])

In [80]:
# Given coordinate and grid information return a location id
def get_id(coordinate, location_list):
    x = coordinate[0]
    y = coordinate[1]
    for location in location_list:
        if x > location['xmin'] and x <= location['xmax'] \
            and y > location['ymin'] and y <= location['ymax']:
            return location['id']
    return False
print(get_id([145, -37.65], location_list))

B2


In [111]:
# Load coordinate and text
twitter_list = [] # Only sotre tweet in the specified location
with open('smallTwitter.json', 'r', encoding = 'utf-8') as f:
    twitter_dic = json.load(f)

for row in twitter_dic['rows']:
    coordinate = row['value']['geometry']['coordinates']
    id = get_id(coordinate, location_list)
    if id:
        text = row['value']['properties']['text']
        twitter_list.append((id, text))
print(twitter_list[0])

('C2', "I'm at @RivaStkilda in St Kilda, VIC https://t.co/kEbicNcmnP")


In [136]:
# Take a single word and a text as input, return multiple matches of the word in that text as a list. 
import re
def allwords(word, text):
    text = re.sub(r'\W\b', ' ',text)
    print(text)
    return re.findall(r'\b%s\b' % word, text, re.I)

print(allwords('good', "Good guidance goOd! all you ide'as people!"))
    

Good guidance goOd! all you ide as people!
['Good', 'goOd']


In [44]:
# The whole text match
score_dict = {}
for location, text in twitter_list:
    for word in word_dict.keys():
        word_list = allwords(word, text)
        length = len(word_list)
        if length > 0:
            score = length * word_dict[word]
            if location in score_dict:
                score_dict[location] += score
            else:
                score_dict[location] = score
        

In [118]:
import re
t = "ha I'm not gooD,not good naïve. I can't stand for hahahathis hard-working! #not Good once-in-a-liFetime."
pattern1 = re.compile(r'can\'t stand|cashing in|cool stuff|does not work|dont like|fed up|green wash|green washing|messing up|no fun|not good\b|not working|right direction|screwed up|some kind', re.I)
pattern2 = re.compile(r'can\'t stand|cashing in|cool stuff|does not work|dont like|fed up|green wash|green washing|messing up|no fun|not good|not working|right direction|screwed up|some kind|[ !?.\"]|\'\W', re.I)
t1 = re.sub(r'\b[!?.,\"]|\'\W', ' ',t)
print(t1)
print(pattern1.findall(t1))
print(pattern2.split(t1))


ha I'm not gooD not good naïve  I can't stand for hahahathis hard-working  #not Good once-in-a-liFetime 
['not gooD', 'not good', "can't stand", 'not Good']
['ha', "I'm", '', '', '', '', 'naïve', '', 'I', '', '', 'for', 'hahahathis', 'hard-working', '', '#', '', 'once-in-a-liFetime', '']


In [85]:
word_dict['naïve']

0